In [1]:
import time
import pandas as pd
import threading
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager  # Importación añadida

def scrape_products():
    # Configura el ChromeDriver utilizando ChromeDriverManager
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    all_products_data = []
    today_date = datetime.now().strftime('%d-%m-%Y')

    for page in range(1, 20):  # Recorremos las páginas del 1 al 20
        driver.get(f"https://www.bubblegummers.cl/582-escolar?page={page}")
        print(f"Procesando la página {page}...")

        try:
            WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.product-miniature')))
        except TimeoutException:
            print(f"Timeout: no se encontraron productos en la página {page}.")
            continue

        # Obtener los enlaces de los productos
        product_links = driver.find_elements(By.CSS_SELECTOR, 'article.product-miniature a.thumbnail')

        if not product_links:
            print(f"No se encontraron productos en la página {page}.")
            continue

        for link in product_links:
            try:
                product_url = link.get_attribute('href')
                driver.execute_script("window.open(arguments[0]);", product_url)  # Abrir el enlace en una nueva pestaña
                driver.switch_to.window(driver.window_handles[-1])  # Cambiar a la nueva pestaña

                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))

                # Extraer los datos del producto
                product_name = driver.find_element(By.CSS_SELECTOR, 'h1.h1[itemprop="name"]').text.strip()
                brand_name = 'Bubble Gummers'

                # EXTRAER EL COLOR
                try:
                    color_element = driver.find_element(By.XPATH, "//li[contains(text(),'Color:')]")
                    color_text = color_element.text.split('Color:')[1].strip()  # Obtener solo el color
                except NoSuchElementException:
                    color_text = ''

                item_id = None
                script = 'return window.dataLayer;'
                data_layer = driver.execute_script(script)

                for item in data_layer:
                    if 'ecommerce' in item and 'items' in item['ecommerce']:
                        for product in item['ecommerce']['items']:
                            if 'item_id' in product:
                                item_id = product['item_id']
                                break
                    if item_id:
                        break

                sku_1 = item_id if item_id else 'N/A'

                original_price = driver.find_element(By.CSS_SELECTOR, 'span.regular-price').text if driver.find_elements(By.CSS_SELECTOR, 'span.regular-price') else ''
                final_price = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="price"]').text if driver.find_elements(By.CSS_SELECTOR, 'span[itemprop="price"]') else ''

                # Inicializa el diccionario del stock, pero sin tallas
                stock_dict = {}

                # Obtener la imagen
                image_element = driver.find_element(By.CSS_SELECTOR, 'img.js-qv-product-cover')
                image_url = image_element.get_attribute('src') 
                image_url_final = image_url if image_url else 'N/A'

                product_data = {
                    'Nombre': product_name,
                    'Marca': brand_name,
                    'Color 1': color_text,
                    'SKU 1': sku_1,
                    'Precio Original': original_price,
                    'Precio Final': final_price,
                    'URL de la Imagen': image_url_final,
                    'URL Página': driver.current_url,
                    'Fecha': today_date,
                }

                product_data.update(stock_dict)
                all_products_data.append(product_data)

                # Guardar los datos en el archivo Excel
                df = pd.DataFrame(all_products_data)
                df.to_excel(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\BURBUJITA_escolar.xlsx', index=False)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])  # Volver a la pestaña principal

                time.sleep(3)

            except ElementClickInterceptedException:
                print(f"Error: Elemento bloqueado para hacer clic en el producto {link.get_attribute('href')}")
            except NoSuchElementException:
                print(f"Error al extraer datos del producto: {link.get_attribute('href')}")
            except TimeoutException:
                print(f"Timeout: al cargar la página del producto {link.get_attribute('href')}")
            except Exception as e:
                print(f"Error inesperado: {e}")

        print(f"Página {page} procesada, moviéndose a la siguiente...")

    driver.quit()

scraping_thread = threading.Thread(target=scrape_products)
scraping_thread.start()

print("El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.")


El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.
Procesando la página 1...
Página 1 procesada, moviéndose a la siguiente...
Procesando la página 2...
